In [1]:
from aicsimageio import AICSImage
import napari
from aicsimageio.readers import CziReader
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#VISUALIZATION
# visualize the image with napari using its numpy array
def visualize_napari(numpy_img: np.ndarray,name):
    """
    :param numpy_img: image to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(numpy_img,name=name)

# visualize different images in the same moment
def visualize_all_list_napari(numpy_img_list: np.ndarray,names):
    """
    :param numpy_img_list: list containing different images to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        for i, img in enumerate(numpy_img_list):
            viewer.add_image(img, name=names[i])


In [3]:
# Load the image
reader = CziReader("TB_sample\extern_Synlab_2156_17_3_MTB.czi")
# Get whole image
smear = reader.get_image_data("MYX", C=0)

In [16]:
visualize_all_list_napari(smear,'e')

C:\Users\matteo\anaconda3\envs\tub\lib\site-packages\napari\_qt\qt_event_loop.py:284: FutureWarning: 
The 'gui_qt()' context manager is deprecated.
If you are running napari from a script, please use 'napari.run()' as follows:

    import napari

    viewer = napari.Viewer()  # no prior setup needed
    # other code using the viewer...
    napari.run()

In IPython or Jupyter, 'napari.run()' is not necessary. napari will automatically
start an interactive event loop for you: 

    import napari
    viewer = napari.Viewer()  # that's it!

  warn(
18-Jan-23 22:13:48 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1512x1104+146+46 (frame: 1534x1160+135+1) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1512x1102+146+46 (frame: 1534x1158+135+1) margins: 11, 45, 11, 11 minimum size: 374x551 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=770,1158 maxtrack=0,0)


In [4]:
def laplacian_blurry(image):
    #compute laplacian of image
    laplacian = cv.Laplacian(image, cv.CV_64F)
    #compute variance of laplacian
    measure=laplacian.var()
    #print(measure)
    return measure/ image.mean()

In [81]:

def get_gradient_magnitude(image):
    #get the gradient magnitude
    sobelx = cv.Sobel(image, cv.CV_64F, 1, 0, ksize=3)
    sobely = cv.Sobel(image, cv.CV_64F, 0, 1, ksize=3)
    magnitude = np.sqrt(sobelx ** 2 + sobely ** 2)
    return magnitude

In [82]:
def sobel(img):
    scale=1
    delta=0
    depth= cv.CV_16S

    grad_x = cv.Sobel(img, depth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
    grad_y = cv.Sobel(img, depth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
    abs_grad_x = cv.convertScaleAbs(grad_x)
    abs_grad_y = cv.convertScaleAbs(grad_y)
    grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    return grad

In [83]:
def sobel_blurry(image):
    mod=get_gradient_magnitude(image)
    mes = mod.sum()
    #take into account that images vary in how much white they have
    return mes

In [84]:
def other_sobel1(image):
    mod = sobel(image)
    mes= mod.sum()
    res = mes
    return res/ image.sum()


In [85]:
def sobel_divided_by_sum_blurry2(image):
    mod=get_gradient_magnitude(image)
    mes = mod.mean()
    #take into account that images vary in how much white they have
    return mes / image.sum()

In [86]:
def is_blurry_fft(image):
    #convert image to grayscale

    #find fourier transform
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    #find mean of magnitude spectrum
    mean = np.mean(magnitude_spectrum)
    #print(mean)
    #if mean is less than 100 then image is blurry
    return mean
#plot spectrum of fft
def plot_spectrum(image):
    #convert image to grayscale

    #find fourier transform
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    #find mean of magnitude spectrum
    mean = np.mean(magnitude_spectrum)
    #print(mean)
    #if mean is less than 100 then image is blurry
    if mean < 200:
        print('blurry')
    else:
        print('not blurry')
    #plot spectrum
    plt.subplot(121),plt.imshow(image, cmap = 'gray')
    plt.title('Input Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(magnitude_spectrum, cmap = 'gray')
    plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
    plt.show()
    #save image

In [5]:
from scipy.cluster.vq import kmeans2

In [6]:
# clustering of the tiles by pixel intensity
def cluster_tiles(smear):
    intensities=np.array([])
    for tile in smear:

        intensities = np.append(intensities,np.count_nonzero(tile>450))

    centroids, labelsz = kmeans2(intensities, 10, minit='points')

    cluster1= np.where(labelsz==0)
    cluster2  = np.where(labelsz == 1)
    cluster3 = np.where(labelsz == 2)
    cluster4 = np.where(labelsz == 3)
    cluster5 = np.where(labelsz == 4)
    cluster6 = np.where(labelsz == 5)
    cluster7 = np.where( labelsz == 6)
    cluster8 = np.where(labelsz == 7)
    cluster9 = np.where(labelsz == 8)
    cluster10 = np.where(labelsz == 9 )
    return [cluster1, cluster2, cluster3, cluster4, cluster5, cluster6, cluster7, cluster8, cluster9, cluster10]




In [7]:
indexes= cluster_tiles(smear)

In [59]:
intensities=np.array([])
for tile in smear:
        tile = rescale_image(tile)
        intensities = np.append(intensities,tile.sum())


In [60]:
for m in range(indexes[2][0].shape[0]):
              print(intensities[indexes[2][0][m]])

12718418.0
12535966.0
12821345.0
12338398.0
15298298.0
12375088.0
12131117.0
12589233.0
47076066.0
59220884.0
116387803.0
107806600.0
15166138.0
22268630.0
10958906.0
137912135.0
59701595.0
27517783.0
27590523.0
13833621.0
212034076.0
15068016.0
16554162.0
71560428.0
152094111.0
95572126.0
98063358.0
49348003.0
45036426.0
28779343.0
24338497.0
26214815.0
11661880.0
16357833.0
142111895.0
75969108.0
30165948.0
26392979.0
17448863.0
78620030.0
117085829.0
88727277.0
178630071.0
177870665.0
143585530.0
108950161.0
61097891.0
68757170.0
85578762.0
29853676.0
22954748.0
20198114.0
20348604.0
15968529.0
73604128.0
125923518.0
69794978.0
26151231.0
23408643.0
24485497.0
133997105.0
109683358.0
242528603.0
83819368.0
36426913.0
12106590.0
33439580.0
37553797.0
45801058.0
265962613.0
137706136.0
94016738.0
82060652.0
71601256.0
32608282.0
19989562.0
30207695.0
11658793.0
77659752.0
117755013.0
214605904.0
125286826.0
91047469.0
45685850.0
122901628.0
142957847.0
71840905.0
73474965.0
56242366.0

In [100]:
del new_smear

In [8]:
measures=[]
means=[]
for o in range(indexes[6][0].shape[0]):
    means.append(smear[indexes[6][0][o]].mean())
nomeans= np.array(means)
realmean=nomeans.mean()
for i in range(indexes[6][0].shape[0]):

    mes = laplacian_blurry(smear[indexes[6][0][i]])
    measures.append(mes)


m = np.array(measures)

m_sorted = np.sort(m)
m_arg = m.argsort()


a = smear[indexes[6][0][np.where(measures == m_sorted[m_arg[0]])]]
b = smear[indexes[6][0][np.where(measures == m_sorted[m_arg[1]])]]

new_smear =  np.stack((a,b), axis=0)

for j in range(2,m.shape[0]):

        print(indexes[6][0][np.where(measures == m_sorted[m_arg[j]])])

        new_smear= np.concatenate((new_smear, [smear[indexes[6][0][np.where(measures == m_sorted[m_arg[j]])]]]), axis= 0)



[1009]
[577]
[522]
[190]
[63]
[122]
[198]
[1111]
[1337]
[1123]
[348]
[1319]
[600]
[1199]
[1064]
[372]
[77]
[367]
[224]
[103]
[514]
[58]
[236]
[1133]
[95]
[624]
[598]
[1110]
[269]
[0]
[419]
[610]
[490]
[75]
[551]
[267]
[599]
[187]
[491]
[1060]
[906]
[952]
[228]
[703]
[410]
[520]
[1061]
[312]
[478]
[360]
[1318]
[516]
[1314]
[244]
[33]
[345]
[1292]
[946]
[303]
[231]
[266]
[515]
[89]
[60]
[477]
[298]
[204]
[617]
[550]
[574]
[288]
[1317]
[167]
[687]
[829]
[1112]
[1125]
[581]
[1114]
[106]
[497]
[197]
[579]
[1079]
[582]
[14]
[9]
[15]
[38]
[56]
[183]
[542]
[622]
[209]
[245]
[203]
[1122]
[201]
[109]
[271]
[147]
[90]
[408]
[199]
[1286]
[406]
[229]
[1057]
[54]
[1284]
[1020]
[536]
[1083]
[1195]
[531]
[195]
[559]
[69]
[307]
[277]
[373]
[1124]
[578]
[289]
[1279]
[571]
[1012]
[949]
[472]
[365]
[517]
[112]
[371]
[6]
[601]
[1082]
[158]


In [9]:
visualize_napari(new_smear,'ff')

C:\Users\matteo\anaconda3\envs\tub\lib\site-packages\napari\_qt\qt_event_loop.py:284: FutureWarning: 
The 'gui_qt()' context manager is deprecated.
If you are running napari from a script, please use 'napari.run()' as follows:

    import napari

    viewer = napari.Viewer()  # no prior setup needed
    # other code using the viewer...
    napari.run()

In IPython or Jupyter, 'napari.run()' is not necessary. napari will automatically
start an interactive event loop for you: 

    import napari
    viewer = napari.Viewer()  # that's it!

  warn(
19-Jan-23 13:38:46 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 2172x1352+782+14 (frame: 2188x1391+774-17) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY2". Resulting geometry: 2172x1187+782+14 (frame: 2188x1226+774-17) margins: 8, 31, 8, 8 minimum size: 374x551 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=390,590 maxtrack=0,0)
19-Jan-23 13:38:46 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unabl

In [58]:
def rescale_image(image):
    image = image.astype(np.float32)
    image -= image.min()
    image *= (255.0/image.max())
    return image.astype(np.uint8)

In [22]:
print(type(smear))
laplacian_list=[]
sobel_list = []
othersobel_list=[]
sobel_sum_list = []
fft_list = []

for tile in smear:
    tile = rescale_image(tile)
    laplacians = laplacian_blurry(tile)
    sobels = sobel_blurry(tile)
    othersobels = other_sobel1(tile)
    sobelsums = sobel_divided_by_sum_blurry2(tile)
    ffts= is_blurry_fft(tile)
    #orders other way around
    laplacian_list.append(laplacians)
    sobel_list.append(sobels)
    othersobel_list.append(othersobels)
    sobel_sum_list.append(sobelsums)
    fft_list.append(ffts)

#get mean of list
l = np.array(laplacian_list)
l_sorted = np.sort(l)
l_arg = l.argsort()

s = np.array(sobel_list)
s_sorted = np.sort(s)
s_arg = s.argsort()

os = np.array(othersobel_list)
os_sorted = np.sort(os)
os_arg = os.argsort(os)

ss = np.array(sobel_sum_list)
ss_sorted = np.sort(ss)
ss_arg = ss.argsort()

ff= np.array(fft_list)
ff_sorted = np.sort(ff)
ff_arg = ff.argsort()



l1 = smear[l_sorted[0]]
l2 = smear[l_sorted[1]]
new_smear_l =  np.stack((l1,l2), axis=0)

s1 = smear[s_sorted[0]]
s2 = smear[s_sorted[1]]
new_smear_s =  np.stack((s1,s2), axis=0)

os1 = smear[os_sorted[0]]
os2 = smear[os_sorted[1]]
new_smear_os =  np.stack((os1,os2), axis=0)

ss1 = smear[ss_sorted[0]]
ss2 = smear[ss_sorted[1]]
new_smear_ss =  np.stack((ss1,ss2), axis=0)

ff1 = smear[ff_sorted[0]]
ff2 = smear[ff_sorted[1]]
new_smear_ff =  np.stack((ff1,ff2), axis=0)

for i in range(2, l.shape[0]):
    if i % 10 ==0:
        new_smear_l = np.concatenate((new_smear_l, [smear[l_arg[i]]]), axis= 0)
        new_smear_s = np.concatenate((new_smear_s, [smear[s_arg[i]]]), axis= 0)
        new_smear_os = np.concatenate((new_smear_os, [smear[os_arg[i]]]), axis= 0)
        new_smear_ss = np.concatenate((new_smear_ss, [smear[ss_arg[i]]]), axis= 0)
        new_smear_ff = np.concatenate((new_smear_ff, [smear[ff_arg[i]]]), axis= 0)



<class 'numpy.ndarray'>


TypeError: 'list' object is not callable

In [ ]:
 print(new_smear.shape)
for i in range(0,len(s),10):
  print(s[i]  )
visualize_napari(new_smear,'ff')
